# ADM HW 3

For this homework, no dataset has been provided. Instead, you have to build your own. Your search engine will run on text documents. So, here we detail the procedure to follow for the data collection. We strongly suggest you work on different modules when implementing the required functions. For example, you may have a crawler.py module, a parser.py module, and a engine.py module: this is a good practice that improves readability in reporting and efficiency in deploying the code. Be careful; you are likely dealing with exceptions and other possible issues!

### 1.1. Get the list of Michelin restaurants
* You should begin by compiling a list of restaurants to include in your document corpus. Specifically, you will focus on web scraping the Michelin Restaurants in Italy. Your task is to collect the URL associated with each restaurant in this list. The output of this step should be a .txt file where each line contains a single restaurant’s URL. By the end, you should have approximately 2,000 restaurants on your list. The number changes daily, so some groups might have different number of restaurants.

---
We will start by loading the relevant libraries first. Then we will try to scrape the *relevant* links on only the first page of the [Michelin website](https://guide.michelin.com/en/it/restaurants/) to test. If the operation goes on successfully, we will scrape the links from the 100 pages!


In [ ]:
# import the relevant libraries
import requests
from bs4 import BeautifulSoup
import os
import re
import lxml
import csv
import random   
import time
from tqdm import tqdm
import asyncio
from tqdm.asyncio import tqdm
import aiohttp
from aiohttp import ClientSession, ClientResponseError
from crawler import *
from myparser import *
import pandas as pd

In [2]:
# setting current working directory to /Users/saifdev/Desktop/ADM/ADM_HW3
os.chdir("/Users/saifdev/Desktop/ADM/ADM_HW3")

Before I begin to build the function `scraping_urls`, which will scrape all the links for the Michelin restaurants in Italy and save them to `urls.txt`, I will start by trying to scrape only the first page to test and practice.

In [ ]:
# A SMALL TEST
url = "https://guide.michelin.com/en/it/restaurants/page/" # url of the Michelin Guide Italy
with open('test.txt', 'w') as file: # open a file to write the links of the restaurants
    current_page = url + "1" # the first page only for testing
    request = requests.get(current_page) # get the first page
    soup = BeautifulSoup(request.content, 'lxml') # parse the content of the first page
    for a in soup.select("div.flex-fill a"): # Ref Keith Galli Y.T. channel :)
        file.write("https://guide.michelin.com"+a.get('href') + '\n')   # write the links of the restaurants in the first page

In [ ]:
# check the number of lines written in the file = 20 (and also manually checked)
with open('test.txt', 'r') as file: # open the file to read the links of the restaurants
    num_lines = len(file.readlines()) # count the number of lines in the file
    result = "20" if num_lines == 20 else "not 20" # check if the number of lines is 20
    print(f"The number of urls scraped is {result}") # print the result. It is indeed 20


The number of urls scraped is 20


It worked as expected. Now we will scrape the urls from the whole 100 pages on the Micheline website using the `scraping_urls` function.

A little note about the `scraping_urls` function:
* It is a function that takes the url of the Michelin Guide Italy webpage and the number of pages to scrape as input and returns a text file containing the URLs of all the restaurants. The function is defined in the crawler.py file.

In [5]:
pages = 100 # set the number of pages to scrape
url = "https://guide.michelin.com/en/it/restaurants/page/" # url of the Michelin Guide Italy
scraping_urls(url=url, pages=pages) # scrape the urls of the restaurants in the Michelin Guide Italy

Scraping Pages: 100%|██████████| 100/100 [03:37<00:00,  2.17s/it]


Checking if the number of urls scraped is approximately 2000

In [6]:
# check the number of urls scraped
with open('urls.txt', 'r') as file:
    num_lines = len(file.readlines())
    print(f"Total Number of restaurants urls scraped is {num_lines}")


Total Number of restaurants urls scraped is 2029


---

### 1.2. Crawl Michelin restaurant pages

##### Once you have all the URLs on the list, you should:

* Download the HTML corresponding to each of the collected URLs.
* After collecting each page, immediately save its HTML in a file. This way, if your program stops for any reason, you will not lose the data collected up to the stopping point.
* Organize the downloaded HTML pages into folders. Each folder will contain the HTML of the restaurants from page 1, page 2, ... of the Michelin restaurant list.
* Tip: Due to the large number of pages to download, consider using methods that can help shorten the process. If you employed a particular process or approach, kindly describe it.

---

Now, we will scrape the information of the restaurants from the urls scraped. To do this, we will use the function load_urls, fetch_and_save, and download_html_in_batches. All these functions are defined in the crawler.py file.

Some information about each of these functions:
* **load_urls**: This function loads the urls of the restaurants from the urls.txt file.
* **fetch_and_save**: This function fetches the html content of the urls and saves the html file into a folder of a particular batch/page.
* **download_html_in_batches**: This function downloads the html content of the urls in batches of 20. The function makes used of `asyncio` and `aiohttp` to download the html content in parallel. This function is used to speed up the process of downloading the html content of the urls due to concurrency.

In [ ]:
url_file = 'urls.txt' # file containing the urls of the restaurants
output_dir = 'michelin_html_batches' # directory to save the sub-directories and the html files
urls = load_urls(url_file) # load the urls of the restaurants

start_time = time.time()
try:
    await download_html_in_batches(urls, output_dir) # using asyncio environment to download the html files in batches
except RuntimeError:
    # Check if there's already an event loop running. Why? Because we can't create a new event loop in a thread that already has
    # an event loop running. Jupyter notebooks often have an event loop running by default. If we try to create a new event loop
    # in a Jupyter notebook, we'll get a RuntimeError. Thus, use asyncio.run if not in a running event loop environment
    asyncio.run(download_html_in_batches(urls, output_dir))

print(f"Finished downloading in {time.time() - start_time} seconds")

Now, we have the html files of the restaurants in the Michelin Guide Italy website downloaded in batches - organised into folders. Now is the time that we start parsing them.

---
### 1.3 Parse downloaded pages

##### At this point, you should have all the HTML documents about the restaurant of interest, and you can start to extract specific information. The list of the information we desire for each restaurant and their format is as follows:

* Restaurant Name (to save as restaurantName): string;
* Address (to save as address): string;
* City (to save as city): string;
* Postal Code (to save as postalCode): string;
* Country (to save as country): string;
* Price Range (to save as priceRange): string;
* Cuisine Type (to save as cuisineType): string;
* Description (to save as description): string;
* Facilities and Services (to save as facilitiesServices): list of strings;
* Accepted Credit Cards (to save as creditCards): list of strings;
* Phone Number (to save as phoneNumber): string;
* URL to the Restaurant Page (to save as website): string. 


---

we have created many functions to parse the html files and extract the information we need. These functions ae in the myparser.py file. Here is the list of functions. The functions are listed below and their names are self-explanatory:  

* `get_restaurant_name(soup)` - taking in soup object and returning the name of the restaurant for example.
* `get_address(soup)`
* `get_city(address)`
* `get_postal_code(address)`
* `get_country(address)`
* `get_price_range(soup)`
* `get_cuisine_type(soup)`
* `get_description(soup)`
* `get_facilities_services(soup)`
* `get_credit_cards(soup)`
* `get_phone_number(soup)`
* `get_website(soup)`

We will test these functions of a random html file form the directories we have created. This will help us to understand the output of these functions and help us evaluate how well they perform.

In [8]:
# Test the parser functions on a particular html file
with open('/Users/saifdev/Desktop/ADM/ADM_HW3/michelin_html_batches/batch_1/20tre.html', 'r', encoding='utf-8') as file:
	content = file.read()
	soup = BeautifulSoup(content, 'lxml')
	name = get_restaurant_name(soup)
	print(f"Restaurant name: {name}")
	address = get_address(soup)
	print(f"Restaurant address: {address}")
	city = get_city(address)
	print(f"Restaurant city: {city}")
	zipcode = get_postal_code(address)
	print(f"Restaurant zipcode: {zipcode}")
	country = get_country(address)
	print(f"Restaurant country: {country}")
	price = get_price_range(soup)
	print(f"Restaurant price range: {price}")
	cuisine = get_cuisine_type(soup)
	print(f"Restaurant cuisine type: {cuisine}")
	description = get_description(soup)
	print(f"Restaurant description: {description}")
	facilities = get_facilities_services(soup)
	print(f"Restaurant facilities and services: {facilities}")
	credit_cards = get_credit_cards(soup)
	print(f"Restaurant credit cards: {credit_cards}")
	phone = get_phone_number(soup)
	print(f"Restaurant phone number: {phone}")
	website = get_website(soup)
	print(f"Restaurant website: {website}")

Restaurant name: 20Tre
Restaurant address: via David Chiossone 20 r, Genoa, 16123, Italy
Restaurant city: Genoa
Restaurant zipcode: 16123
Restaurant country: Italy
Restaurant price range: €€
Restaurant cuisine type: Farm to table, Modern Cuisine
Restaurant description: Situated in the heart of Genoa’s historic centre, this contemporary-style restaurant focuses on just a few dishes, almost all fish-based, presented in a very modern style and in generous portions. Seasonal ingredients and market-fresh produce are the guiding philosophy here.
Restaurant facilities and services: ['Air conditioning']
Restaurant credit cards: ['amex', 'dinersclub', 'mastercard', 'visa']
Restaurant phone number: +39 010 247 6191
Restaurant website: https://www.ristorante20tregenova.it/


Our Parser functions work perfectly. Let's employ them now to get the required data from all the html files we have stored. Then, we will gather the data into a csv file which we can later read into the Pandas DataFrame.

To do this, we will use the following functions stored in myparser.py:
`extract_restaurant_data`, `save_restaurant_data_to_csv`, `forming_a_csv`

A brief explanation of the functions:
* `extract_restaurant_data` - This function takes the html content of a restaurant page and extracts the relevant data using the parser functions.
* `save_restaurant_data_to_csv` - This function takes the extracted data and saves it to a csv file.
* `forming_a_csv` - This function takes the html files stored in the directory and extracts the data from each file and saves it to a csv file.


In [ ]:
# form a csv file with the information of the restaurants using extract_restaurant_data function and the save_restaurant_data_to_csv function
forming_a_csv()

Data saved to michelin_restaurant_data.csv


Finally, we have the csv file with the information of the restaurants using the aforementioned functions. The csv file is named *"michelin_restaurants.csv"*. We can now load the csv file using `pd.read_csv` and perform the required tasks.

In [15]:
data = pd.read_csv('michelin_restaurant_data.csv')
data.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,Locanda Radici,"SP 21, contrada San Vincenzo, Melizzano, 82030...",Melizzano,82030.0,Italy,€€,"Modern Cuisine, Campanian",A rustic restaurant with contemporary Mediterr...,Air conditioning; Car park; Garden or park; Gr...,amex; mastercard; visa,+39 0824 944506,https://www.locandaradici.it/
1,Posta,"viale Vittorio Veneto 169, Sant'Omobono Terme,...",Sant'Omobono Terme,24038.0,Italy,€€€,Italian,"Situated in the Imagna valley, this welcoming,...",Air conditioning; Wheelchair access,amex; dinersclub; mastercard; visa,+39 035 851134,https://www.frosioristoranti.it
2,Hostaria Baccofurore,"via G.B. Lama 9, Furore, 84010, Italy",Furore,84010.0,Italy,€€,"Regional Cuisine, Farm to table",Patience is needed to get to this restaurant f...,Air conditioning; Car park; Garden or park; Gr...,amex; dinersclub; jcb; maestrocard; mastercard...,+39 089 830360,https://www.baccofurore.it/
3,Nni Lausta,"via Risorgimento 188, Santa Marina Salina, 980...",Santa Marina Salina,98050.0,Italy,€€,"Seafood, Seasonal Cuisine","Fish plays the starring role here, where the a...",Terrace,amex; jcb; maestrocard; mastercard; visa,+39 090 984 3486,NaN
4,Osteria de Börg,"via Forzieri 12, Rimini, 47921, Italy",Rimini,47921.0,Italy,€,"Cuisine from Romagna, Traditional Cuisine",Borgo San Giuliano is situated just a stroll f...,Terrace,amex; maestrocard; mastercard; visa,+39 0541 56074,https://www.osteriadeborg.it/


In [14]:
print(f"Checking the shape of the data: It should be {data.shape}")

Checking the shape of the data: It should be (2029, 12)


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   restaurantName      2029 non-null   object 
 1   address             2029 non-null   object 
 2   city                2000 non-null   object 
 3   postalCode          1983 non-null   float64
 4   country             2029 non-null   object 
 5   priceRange          1983 non-null   object 
 6   cuisineType         2029 non-null   object 
 7   description         2029 non-null   object 
 8   facilitiesServices  1959 non-null   object 
 9   creditCards         1979 non-null   object 
 10  phoneNumber         1983 non-null   object 
 11  website             1875 non-null   object 
dtypes: float64(1), object(11)
memory usage: 190.3+ KB


# 2.0 Preprocessing
## 2.0.0) Preprocessing the text

Before building the search engine, you must clean and prepare the text in each restaurant’s description.

We will:

+ Remove stopwords.
+ Remove punctuation.
+ Apply stemming.
+ Perform any other necessary cleaning to improve search accuracy.
>For this, we use the nltk library.

In [1]:
!pip install nltk

     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     -------------------------------------- 41.5/41.5 kB 496.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.5 MB 1.7 MB/s eta 0:00:01
   ---------- ----------------------------- 0.4/1.5 MB 2.1 MB/s eta 0:00:01
   -------------- ------------------------- 0.6/1.5 MB 1.9 MB/s eta 0:00:01
   ----------------------- ---------------- 0.9/1.5 MB 2.3 MB/s eta 0:00:01
   -------------------------- ------------- 1.0/1.5 MB 2.2 MB/s eta 0:00:01
   -------------------------------- ------- 1.2/1.5 MB 2.4 MB/s eta 0:00:01
   -------------------------------------- - 1.4/1.5 MB 2.5 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/273.6 kB ? eta -:--:--
   ---------------------------------------- 273.6/273.6 kB 8.5 MB/s eta 0:00:00
   -----------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
# importing necessary libraries
import nltk
nltk.data.path = ['C:/Users/Val/AppData/Roaming/nltk_data']
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string

# Download necessary NLTK data files
nltk.download('stopwords')
nltk.download('punkt')

print(nltk.data.path)

print(nltk.data.find('corpora/stopwords.zip'))
print(nltk.data.find('tokenizers/punkt'))

['C:/Users/Val/AppData/Roaming/nltk_data']
C:\Users\Val\AppData\Roaming\nltk_data\corpora\stopwords.zip
C:\Users\Val\AppData\Roaming\nltk_data\tokenizers\punkt


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Val\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Val\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [46]:
stop_words = set(stopwords.words('english')) 
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocessing(string):

    #ensure text is a string (in case of NaN or other data types)
    if not isinstance(string, str):
        return ""
    
    #to lower case
    string = string.lower()
    # Remove extra whitespace
    string = re.sub(r'\s+', ' ', string).strip()
    #handling compound words with '-' to treat them as separate tokens
    string = re.sub(r'-', ' ', string) 

    #tokenize
    tokens = word_tokenize(string)
    #remove punctuation: keep only aplhanumeric tokens
    tokens = [token for token in tokens if token.isalnum()]
    #remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    #lemmitization before stemming
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    #stemming to reduce words to their root in order to reduce our vocabulary size, improve search accuracy and speed up search process
    tokens = [stemmer.stem(token) for token in tokens]

    #return the cleaned string
    preprocessed_string = ' '.join(tokens)
    return preprocessed_string

In [47]:
data['cleaned_description'] = data['description'].apply(preprocessing)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:/Users/Val/AppData/Roaming/nltk_data'
**********************************************************************
